In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

%matplotlib inline

x = tf.constant([5, 3, 8], dtype=tf.float32, name="variable_a")
x = tf.Variable(2.0, dtype=tf.float32, name="my_variable")
# x.assign(new_value)
# x.assign_add(value_to_be_added)
# x.assign_sub(value_to_be_subtracted


# Point operation
# tf.add allows to add the components of a tensor
# tf.multiply allows us to multiply the components of a tensor
# tf.subtract allow us to substract the components of a tensor
# tf.math.*

# Training

In [ ]:
import tf 

def loss_mse(X,Y,w0,w1):
    Y_hat = X*w0 + w1
    return tf.reduce_mean(tf.square(Y_hat - Y))  # alternate: tf.sub

def compute_gradient(X, Y, w0, w1):
    with tf.GradientTape as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

# TRAINING    
STEPS = 1000
LEARNING_RATE = 0.02
MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

for step in range(0, STEPS + 1):
    dw0, dw1 = compute_gradients(X,Y,w0,w1)
    w0.assign_sub(dw0*LEARNING_RATE) # GRADIENT -> DIRECTION, LR -> THE AMOUNT OF MOVEMENT
    w1.assign_sub(dw1*LEARNING_RATE) 

    if step % 100 == 0:
        loss = loss_mse(X,Y,w0,w1)  # TODO: Your code goes here.
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))


# Bonus

In [ ]:
def make_features(X):
    f1 = tf.ones_like(X)  # Bias.
    f2 = X
    f3 = tf.square(X)
    f4 = tf.sqrt(X)
    f5 = tf.exp(X)
    return tf.stack([f1, f2, f3, f4, f5], axis=1)

def predict(X, W):
    return tf.squeeze(X @ W, -1)

X = tf.constant(np.linspace(0, 2, 1000), dtype=tf.float32)
Y = X * tf.exp(-(X ** 2))

plt.plot(X, Y)

STEPS = 2000
LEARNING_RATE = 0.02
Xf = make_features(X)
n_weights = Xf.shape[1]
W = tf.Variable(np.zeros((n_weights, 1)), dtype=tf.float32)

# PLOTTING
steps, losses = [], []
plt.figure()
for step in range(1, STEPS + 1):
    dW = compute_gradients(X, Y, W)
    W.assign_sub(dW * LEARNING_RATE)
    if step % 100 == 0:
        loss = loss_mse(Xf, Y, W)
        steps.append(step)
        losses.append(loss)
        plt.clf()
        plt.plot(steps, losses)
print(f"STEP: {STEPS} MSE: {loss_mse(Xf, Y, W)}")
plt.figure()
plt.plot(X, Y, label="actual")
plt.plot(X, predict(Xf, W), label="predicted")
plt.legend()

# TF Dataset API

# Keras Sequential API
- feature columns for feature engineering

In [ ]:
import datetime
import os
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import aiplatform
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, DenseFeatures, Dropout
from tensorflow.keras.models import Sequential

print(tf.__version__)
%matplotlib inline

# taxi dataset
def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    for unwanted_col in UNWANTED_COLS: features.pop(unwanted_col)
    return features, label


def create_dataset(pattern, batch_size=1, mode="eval"):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS      # 
    )
    dataset = dataset.map(features_and_labels)
    if mode == "train": dataset = dataset.shuffle(buffer_size=1000).repeat()
    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

CSV_COLUMNS = [
    "fare_amount",
    "pickup_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "key",
]
LABEL_COLUMN = "fare_amount"
DEFAULTS = [[0.0], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0], ["na"]]   # COLUMNS DEFAULTS
UNWANTED_COLS = ["pickup_datetime", "key"]

INPUT_COLS = [
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
]

feature_columns =  [
    tf.feature_column.numeric_column(key=key)
    for key in INPUT_COLS
]
# Build a keras DNN model using Sequential API
model =  keras.Sequential([
    DenseFeatures(feature_columns=feature_columns),
    Dropout(0.5),
    Dense(128, activation="relu", name="h1"),
    Dropout(0.4),
    Dense(64, activation="relu", name="h2"),
    Dropout(0.3),
    Dense(32, activation="relu", name="h3"),
    Dense(1, activation="linear", name="output")
])

# COMPILE
def rmse(y_true, y_pred):
    return  tf.reduce_mean(tf.square(tf.subtract(y_pred, y_true)))

model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

# TRAINING
# .fit() for training a model for a fixed number of epochs (iterations on a dataset).
# .fit_generat
TRAIN_BATCH_SIZE = 1000
NUM_TRAIN_EXAMPLES = 10000 * 5  # training dataset will repeat, wrap around
NUM_EVALS = 50  # how many times to evaluate
NUM_EVAL_EXAMPLES = 10000  # enough to get a reasonable sample

trainds = create_dataset(
    pattern="../data/taxi-train*", batch_size=TRAIN_BATCH_SIZE, mode="train"
)
evalds = create_dataset(
    pattern="../data/taxi-valid*", batch_size=1000, mode="eval"
).take(NUM_EVAL_EXAMPLES // 1000)or() for training a model on data yielded batch-by-batch by a generator
# .train_on_batch() runs a single gradient update on a single batch of data.

